## Enter positions

In [1]:
query = [
    'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 1 4', # opening
    'rn1qkb1r/1b2pppp/p2p1n2/1p4B1/3NP3/2N2Q2/PPP2PPP/R3KB1R w KQkq - 2 8', # opening
    'rnbqk2r/4bppp/p2ppn2/1p4B1/3NP3/2N5/PPPQ1PPP/2KR1B1R w kq - 0 9', # opening
    'r2qkb1r/pbpp1ppp/1pn1pn2/8/2PP4/5NP1/PP2PPBP/RNBQ1RK1 b kq - 0 6', # unusual opening
    '1r1qkb1r/pBpn1ppp/4b2n/8/8/4Q3/PP1PPP1P/RNB1K1NR w KQk - 3 9', # unusual opening
    'r1bq1rk1/pp1nbppp/2p1pn2/3P2B1/3P4/2NBPN2/PP3PPP/R2QK2R b KQ - 0 8', # queens gambit
    '5rk1/3nqppp/r1p2n2/p3p1B1/4P3/2P2N2/P1Q2PPP/3R1RK1 w - - 0 15', # middle game
    'r3k2r/pbp1qppp/1pn1pn2/3pP3/Q1P5/P1N3P1/1P2PPBP/R1B2RK1 b kq - 0 13', # middle game
    'r4rk1/1bq1bppp/p3p3/2npP3/1p1N1PPP/1B2Q3/PPP1N3/1K1R3R w - - 1 20', # middle game
    'r1b2rk1/ppq3p1/2p1p1Bp/2n1P3/2p2P2/2N3P1/1PQ1P2P/R4RK1 b - - 1 16', # middle game
    '1r1q1b1r/p1p1kppp/2B1b3/2n2n2/8/8/PP1PPPQP/RNB1K1NR b KQ - 10 12', # king in the middle
    'r1bqr1kb/p3pp2/np1p2pB/3P2N1/4P1nP/2N5/PP1QB3/R3K2R w KQ - 0 17', # attack
    'r2qk2r/4bpp1/p1b1p2p/3p3P/1p3P2/3B2P1/PPPQ1N2/2KR3R w kq - 0 18', # typical attack
    'r2qk2r/1p1n1pp1/p1pbp3/8/PPQP3p/2N1PNP1/5PP1/R4RK1 w kq - 0 15', # attack
    'rr4k1/4bpPp/4p2Q/3pq2N/1p1N2PP/p1Pb4/PP6/1K1RR3 w - - 2 30', # attack
    'r4r2/4p1k1/p4npQ/qppPpp2/P1P1P3/2N2P2/1P6/R3K2R b KQ - 1 21', # attack
    '4kr2/5p2/4p2p/7P/3QNP2/1p6/qK6/r2R2R1 w - - 3 31', # attack, check
    '2kr4/1p1n1pp1/p1p1p3/8/PPQP4/2N1P1q1/8/R4R1K w - - 2 20', # open line
    '5rk1/3nqp2/1rp4p/p3p1p1/2Q1P3/2PR1Nn1/P4PPP/5RK1 w - - 0 20', # capture
    '2r2qrk/4p3/pp1pP2b/3N4/8/3Q4/PP6/1K5R b - - 0 32', # material imbalance
    '4kr2/5p2/4p2p/q6P/3KNP2/1p6/8/3R2R1 b - - 0 34', # material imbalance
    '8/p2R2pk/1p2p2p/4P3/4PB1P/P3K1P1/1P2N3/1q6 w - - 9 39', # material imbalance
    '7k/5Bp1/1r2p1KP/2p1P1P1/2P5/4P3/8/8 b - - 0 49', # material imbalance
    'r1b2rk1/pp1nqppp/2p5/8/3PP3/3B1N2/P4PPP/R2Q1RK1 b - - 0 13', # hanging pawns
    'r2qkb1r/pppn1ppp/4p3/3p4/3P4/3QPNP1/PPP2PP1/RN3RK1 b kq - 0 9', # closed position
    '8/8/2p2n1P/p3p2N/6p1/4K1P1/Pk6/8 b - - 2 37', # endgame
    '8/8/2p2K1P/8/8/6P1/pk6/8 b - - 0 44', # endgame
    '8/7p/8/1K6/3k4/2p5/P6P/8 w - - 0 49', # endgame
]
search_results = 20

## Load and search database of bitboards

Instead of loading indices and searching yourself you can also check out the 'Compare retireved results' section and unpickle precompiled searches!

In [2]:
from chesspos.binary_index import index_load, index_query_positions

In [3]:
filepath = "/media/pafrank/Backup/other/Chess/lichess/indices/index_2015"
bb_index = index_load(filepath, is_binary=True)
print(f"The database you loaded contains {round(bb_index.ntotal/1.e6,3)} million positions")

The database you loaded contains 46.728 million positions


In [4]:
%%time
dist, reconstructed = index_query_positions(query, bb_index, input_format='fen', output_format='fen',
                                            num_results=search_results)

CPU times: user 3min 46s, sys: 827 ms, total: 3min 47s
Wall time: 58.6 s


In [6]:
import pickle
pickle.dump(dist, open('examples/example_bitboard_distances.p','wb'))
pickle.dump(reconstructed, open('examples/example_bitboards.p','wb'))

## Load and search database of embeddings

In [2]:
import json
import faiss
import numpy as np
import tensorflow as tf

from chesspos.convert import bitboard_to_board
from chesspos.binary_index import board_to_bitboard
from chesspos.utils import files_from_directory
import chesspos.embedding_index as iemb

In [3]:
# prepare everything for search
encoder_path = "./models/deep64/model_encoder.h5"
decoder_path ="./models/deep64/model_decoder.h5"
embedding_path = "./indices/all_bitboards"
index_file_without_ending = "./indices/all_d64/PCA16,SQ4"
# load the index
table_dict = json.load( open( f"{index_file_without_ending}.json" ) )
index = faiss.read_index(f"{index_file_without_ending}.faiss")
query = np.asarray(query)
print(f"The database you loaded contains {round(index.ntotal/1.e6,3)} million positions")

The database you loaded contains 907.544 million positions


In [4]:
%%time
D, I, E = iemb.index_query_positions(query, index, encoder_path,
                                     input_format='fen', num_results=search_results)

CPU times: user 49min 14s, sys: 1min 27s, total: 50min 41s
Wall time: 14min 14s


In [5]:
%%time
# retrieve the belonging bitboards
file, table, offset = iemb.location_from_index(I, table_dict)
bb_table = iemb.manipulate_prefix(table, "position")
bb_file = iemb.manipulate_prefix(file, f"{embedding_path}/lichess_db_standard_rated")

embedding_bitboards = iemb.retrieve_elements_from_file(bb_file, bb_table, offset)
bb_shape = embedding_bitboards.shape
print(embedding_bitboards.shape, embedding_bitboards.dtype)

[   774324   1550504   2328781 ... 906804818 906877453 907544126]
(28, 20, 773) bool
CPU times: user 11.4 s, sys: 714 ms, total: 12.1 s
Wall time: 37 s


In [6]:
# convert bitboards to fen
def fen_converter(bb):
    bb = bb.astype(bool)
    board = bitboard_to_board(bb) 
    return board.fen()

bitboards_fen = [[None for _ in range(bb_shape[1])] for _ in range(bb_shape[0])]
for i in range(bb_shape[0]):
    for j in range(bb_shape[1]):
        bitboards_fen[i][j] = fen_converter(embedding_bitboards[i][j])
print(len(bitboards_fen),len(bitboards_fen[0]), bitboards_fen)

28 20 [['rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1', 'rnbqkb1r/ppp1pp1p/3p1np1/8/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 1'

In [7]:
import pickle
pickle.dump(D, open('examples/example_embedding_distances.p','wb'))
pickle.dump(bitboards_fen, open('examples/example_embeddings.p','wb'))

## Compare the retrieved results

If you haven't run the searches yourself, then execute the pickle.load cells below for some precompiled searches

| Index          | Positions | RAM    | Disk    |
|----------------|-----------|--------|---------|
| 2015 bitboards | 46.7 M    | 5.2 Gb | 5.9Gb   |
| all embeddings | 907.5 M   | 10.4 Gb | 11.9 Gb |

### Bitboards

In [ ]:
# optionally load example results
import pickle
dist = pickle.load(open('examples/example_bitboard_distances.p','rb'))
reconstructed = pickle.load(open('examples/example_bitboards.p','rb'))

In [ ]:
from IPython.display import HTML
html = '''<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous"><table>'''
for i in range(len(reconstructed)):
    html += f'''<tr><td>Your Query Position {i}</td><td><span>The (Hamming) distance between query and </span><select id="mySelect{i}" onchange="myFunction{i}()">'''
    for j in range(search_results):
        html += f'''<option value='{reconstructed[i][j]}|{dist[i][j]}'>Similar Position {j}</option>'''
    html += f'''</select><span> is </span><span id="dist{i}">0</span><span>.</span></td></tr><tr><td><div id="query{i}" style="width: 400px"></div></td><td><div id="myBoard{i}" style="width: 400px"></div></td></tr>'''
html += '''</table><script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script><script>'''
for i in range(len(reconstructed)):
    html += f'''var pos{i} = document.getElementById("mySelect{i}").value;var board{i} = Chessboard('myBoard{i}',{{showNotation: false}});var query{i} = Chessboard('query{i}',{{position: '{query[i]}',showNotation: false}});function myFunction{i}() {{var infos = document.getElementById("mySelect{i}").value;var position = infos.split("|")[0];var distance = infos.split("|")[1];board{i}.position(position);document.getElementById("dist{i}").innerHTML = distance;}}'''
html += '''</script>'''
HTML(html)

### Embeddings

In [ ]:
import pickle
D = pickle.load(D, open('examples/example_embedding_distances.p','rb'))
bitboards_fen = pickle.load(open('examples/example_embeddings.p','rb'))

In [ ]:
from IPython.display import HTML
html = '''<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous"><table>'''
for i in range(len(bitboards_fen)):
    html += f'''<tr><td>Your Query Position {i}</td><td><span>Distance between query and </span><select id="mySelectEmb{i}" onchange="myFunctionEmb{i}()">'''
    for j in range(search_results):
        html += f'''<option value='{bitboards_fen[i][j]}|{D[i][j]}'>Similar Position {j}</option>'''
    html += f'''</select><span> is </span><span id="distEmb{i}">0</span><span>.</span></td></tr><tr><td><div id="queryEmb{i}" style="width: 400px"></div></td><td><div id="myBoardEmb{i}" style="width: 400px"></div></td></tr>'''
html += '''</table><script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script><script>'''
for i in range(len(bitboards_fen)):
    html += f'''var pos{i} = document.getElementById("mySelectEmb{i}").value;var board{i} = Chessboard('myBoardEmb{i}',{{showNotation: false}});var query{i} = Chessboard('queryEmb{i}',{{position: '{query[i]}',showNotation: false}});function myFunctionEmb{i}() {{var infos = document.getElementById("mySelectEmb{i}").value;var position = infos.split("|")[0];var distance = parseFloat(infos.split("|")[1]).toFixed(2);board{i}.position(position);document.getElementById("distEmb{i}").innerHTML = distance;}}'''
html += '''</script>'''
HTML(html)

## Concluding remaks

Here we looked at searches on a larger bitboard database in comparison to searches on a larger embedding database.

The constraining factor is the available RAM and here we are constrained to roughly 10 GB. The performance of embedding search is ambivalent. Although surprisingly good in many cases it fails in endgame positions and in positions with imbalanced material. Both may partially be attributed to the lack of similar games in the training/serach set. Where available embedding search confidently matches the exact position, however sometimes positions are different and difficult to judge.

The results suggests that a combined bitboard/embedding search might yield superior results. But remark that the embedding search results are already good and that it's memory footprint is tiny.